In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from scipy.misc import imresize

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import backend as K
 
 
seed = 42
epochs = 1
records_per_class = 100
 
# We take only 2 classes from CIFAR10 and a very small sample to intentionally overfit the model.
# We will also use the same data for train/test and expect that Keras will give the same accuracy.
(x, y), _ = cifar10.load_data()
 
def filter_resize(category):
   # We do the preprocessing here instead in the Generator to get around a bug on Keras 2.1.5.
   return [preprocess_input(imresize(img, (224,224)).astype('float')) for img in x[y.flatten()==category][:records_per_class]]
 
x = np.stack(filter_resize(3)+filter_resize(5))
records_per_class = x.shape[0] // 2
y = np.array([[1,0]]*records_per_class + [[0,1]]*records_per_class)
 
 

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


In [2]:
# We will use a pre-trained model and finetune the top layers.
np.random.seed(seed)
def mymodel():
    with K.learning_phase_scope(0):
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    # base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    c = keras.layers.Conv2D(64, (3, 3), padding='same')(base_model.output)
    
#     b = keras.layers.BatchNormalization(momentum=0.9)(c, training=True) # this way save state in h5 file
#     with K.learning_phase_scope(1): # this way don't save state in h5 file
#         b = keras.layers.BatchNormalization(momentum=0.99)(c)
    b = keras.layers.BatchNormalization(momentum=0.99)(c)
    r = keras.layers.ReLU()(b)
    l = Flatten()(r)
    # l = Flatten()(base_model.output)
    predictions = Dense(2, activation='softmax')(l)
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in model.layers[:140]:
       layer.trainable = False
    for layer in model.layers[140:]:
       layer.trainable = True

    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = mymodel()
model.fit_generator(ImageDataGenerator().flow(x, y, seed=42), epochs=epochs, validation_data=ImageDataGenerator().flow(x, y, seed=42))
 
# Store the model on disk
model.save('tmp.h5')
 
 

/root/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


7/7 [==============================] - 5s 745ms/step - loss: 0.7640 - accuracy: 0.5700 - val_loss: 0.2919 - val_accuracy: 0.8300


In [3]:
model.layers[-4].get_weights()

[array([0.9999163 , 0.99998325, 0.9999877 , 0.99989146, 0.99992657,
        0.99976426, 1.0000074 , 1.0000465 , 0.99999577, 0.99994373,
        1.0000684 , 0.9998759 , 1.0001304 , 1.0000405 , 1.0000027 ,
        1.0000796 , 0.9998667 , 0.9999276 , 1.0000191 , 0.9999217 ,
        0.9999243 , 0.99997   , 0.99995923, 1.0001112 , 1.000051  ,
        1.0000095 , 0.99995774, 1.0001501 , 0.99995637, 0.9999779 ,
        0.9999396 , 1.000062  , 1.0000038 , 0.9998946 , 0.99996436,
        0.9999806 , 1.0000055 , 0.99988204, 0.999955  , 1.0000309 ,
        1.0001107 , 1.0000203 , 1.0001026 , 1.0000283 , 1.0000458 ,
        1.0000329 , 0.99991006, 0.99997276, 0.99992627, 1.0000035 ,
        1.0000618 , 0.9998756 , 0.99981934, 0.99999666, 1.0000732 ,
        0.99993163, 0.99993765, 0.9999517 , 0.9999914 , 0.9999175 ,
        0.9999729 , 0.99992675, 1.0000046 , 0.9997653 ], dtype=float32),
 array([-3.88650333e-05,  6.52829840e-05, -4.68558901e-05, -6.33832460e-05,
        -2.05347988e-05, -1.4121246

In [5]:
def mymodel(t1, t2):
    with K.learning_phase_scope(t1):
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    # base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    c = keras.layers.Conv2D(64, (3, 3), padding='same')(base_model.output)
    b = keras.layers.BatchNormalization(momentum=0.9)(c, training=t2)
#     with K.learning_phase_scope(t2):
#         b = keras.layers.BatchNormalization(momentum=0.9)(c)
    r = keras.layers.ReLU()(b)
    l = Flatten()(r)
    # l = Flatten()(base_model.output)
    predictions = Dense(2, activation='softmax')(l)
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in model.layers[:140]:
       layer.trainable = False
    for layer in model.layers[140:]:
       layer.trainable = True

    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# the new bn layer momentun 0.99 so large that moving_mean&&var can not learn well by small train_steps(epoch).
print('LEARNING_PHASE 0,0')
K.clear_session()
model = mymodel(0,0)
model.load_weights('tmp.h5')
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))

# would get better result because using mean&&var(compute by mini-batch) from the distribution of new dataset.
print('LEARNING_PHASE 0,1')
K.clear_session()
model = mymodel(0,1)
model.load_weights('tmp.h5')
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))

print('LEARNING_PHASE 1,0') 
K.clear_session()
model = mymodel(1,0)
model.load_weights('tmp.h5')
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))

print('LEARNING_PHASE 1,1')
K.clear_session()
model = mymodel(1,1)
model.load_weights('tmp.h5')
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))

print('DYNAMIC LEARNING_PHASE the same as the saved_h5_model')
K.clear_session()
model = load_model('tmp.h5')
# This accuracy should match exactly the one of the validation set on the last iteration.
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))


LEARNING_PHASE 0,0
[0.2918673705841814, 0.83]
LEARNING_PHASE 0,1
[0.34467991814017296, 0.875]
LEARNING_PHASE 1,0
[0.8779313734599522, 0.575]
LEARNING_PHASE 1,1
[0.7295647859573364, 0.59]
DYNAMIC LEARNING_PHASE the same as the saved_h5_model
[0.2918673705841814, 0.83]
